# Quokka

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [20]:
df = pd.read_csv('datasets/forestfires.csv')

In [21]:
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [22]:
df.count()

X        517
Y        517
month    517
day      517
FFMC     517
DMC      517
DC       517
ISI      517
temp     517
RH       517
wind     517
rain     517
area     517
dtype: int64

In [23]:
df.drop(['X', 'Y', 'day'], axis=1, inplace=True)

In [24]:
# Dictionary mapping month names to month numbers
month_to_number = {
    'jan': 1,
    'feb': 2,
    'mar': 3,
    'apr': 4,
    'may': 5,
    'jun': 6,
    'jul': 7,
    'aug': 8,
    'sep': 9,
    'oct': 10,
    'nov': 11,
    'dec': 12
}

# Convert month names to month numbers using map() function
df['month'] = df['month'].map(month_to_number)


In [25]:
X = df[['month', 'temp', 'RH', 'wind', 'rain']]
Y = (df['area'] > 0).astype(float)

In [26]:
X.shape

(517, 5)

In [27]:
X.dtypes

month      int64
temp     float64
RH         int64
wind     float64
rain     float64
dtype: object

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5)


In [29]:
X_test


,month,temp,RH,wind,rain
423,9,22.3,48,4.0,0.0
105,3,12.7,52,6.3,0.0
49,3,11.0,46,5.8,0.0
358,9,19.7,39,2.7,0.0
302,6,19.2,38,4.5,0.0
...,...,...,...,...,...
379,1,5.2,100,0.9,0.0
291,7,25.3,39,0.9,0.0
75,2,6.7,79,3.1,0.0
426,8,20.4,56,2.2,0.0


In [30]:
model = Sequential(
    [
        Dense(128, activation='relu', input_shape=(5,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)


In [31]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               768       
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 11,137
Trainable params: 11,137
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
9/9 [==============================] - 3s 92ms/step - loss: 0.8982 - accuracy: 0.5000 - val_loss: 0.7535 - val_accuracy: 0.5290
Epoch 2/100
9/9 [==============================] - 0s 22ms/step - loss: 0.7334 - accuracy: 0.5078 - val_loss: 0.6910 - val_accuracy: 0.5212
Epoch 3/100
9/9 [==============================] - 0s 19ms/step - loss: 0.7281 - accuracy: 0.5233 - val_loss: 0.7198 - val_accuracy: 0.5135
Epoch 4/100
9/9 [==============================] - 0s 24ms/step - loss: 0.7076 - accuracy: 0.5388 - val_loss: 0.7221 - val_accuracy: 0.4710
Epoch 5/100
9/9 [==============================] - 0s 23ms/step - loss: 0.6943 - accuracy: 0.5736 - val_loss: 0.7243 - val_accuracy: 0.5483
Epoch 6/100
9/9 [==============================] - 0s 24ms/step - loss: 0.6870 - accuracy: 0.5698 - val_loss: 0.7406 - val_accuracy: 0.4903
Epoch 7/100
9/9 [==============================] - 0s 32ms/step - loss: 0.6909 - accuracy: 0.5698 - val_loss: 0.7066 - val_accuracy: 0.5290
Epoch 8/100
9/9 [===

In [34]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss:.2f}')
print(f'Accuracy: {accuracy:.2f}')

# Make predictions on new data
# [[month, temp, RH, wind, rain]]
# some test samples:
# [[6, 40, 20, 4.3,0.0]]
# [[12, 10, 70, 0,0]]
# [[1, 20, 50, 2.2,0.0]]

new_data = [[6, 40, 20, 4.3,0.0]]
predictions = model.predict(new_data)

# prediction in range 0 - 1`
print(f'Chance of forest fire: {predictions[0][0]:3f}')

9/9 [==============================] - 0s 7ms/step - loss: 0.7416 - accuracy: 0.5174
Loss: 0.74
Accuracy: 0.52
1/1 [==============================] - 0s 349ms/step
Chance of forest fire: 0.324786


In [35]:
## multiple prediction
import random as rand
new_data = [(rand.randint(1,12), rand.randint(0,40), rand.randint(0,100), rand.randint(0,10), rand.randint(0,10)) for i in range(100)]

predictions = model.predict(new_data)
for i in range(len(new_data)):
    print(f"month: {new_data[i][0]} temp: {new_data[i][1]} RH: {new_data[i][2]} wind: {new_data[i][3]} rain: {new_data[i][4]}\nchance of forest fire: {predictions[i][0]:3f}\n")

4/4 [==============================] - 0s 11ms/step
month: 10 temp: 30 RH: 32 wind: 3 rain: 0
chance of forest fire: 0.727301

month: 2 temp: 2 RH: 37 wind: 9 rain: 3
chance of forest fire: 0.536671

month: 5 temp: 8 RH: 55 wind: 2 rain: 7
chance of forest fire: 0.051588

month: 12 temp: 7 RH: 64 wind: 9 rain: 2
chance of forest fire: 0.531109

month: 12 temp: 30 RH: 21 wind: 5 rain: 7
chance of forest fire: 0.200098

month: 12 temp: 8 RH: 83 wind: 9 rain: 5
chance of forest fire: 0.189282

month: 7 temp: 25 RH: 67 wind: 8 rain: 6
chance of forest fire: 0.047358

month: 11 temp: 27 RH: 84 wind: 8 rain: 2
chance of forest fire: 0.166461

month: 1 temp: 0 RH: 94 wind: 7 rain: 7
chance of forest fire: 0.241207

month: 2 temp: 9 RH: 87 wind: 2 rain: 6
chance of forest fire: 0.043764

month: 6 temp: 12 RH: 20 wind: 2 rain: 4
chance of forest fire: 0.291662

month: 11 temp: 9 RH: 67 wind: 3 rain: 5
chance of forest fire: 0.083334

month: 8 temp: 14 RH: 40 wind: 4 rain: 8
chance of forest fir

In [36]:
#model.save('models/quokkaForesterV1.0.h5')

In [46]:
from keras.models import load_model
saved_model = load_model('models/quokkaForesterV1.0.h5')
import random as rand
new_data = [(rand.randint(1,12), rand.randint(0,40), rand.randint(0,100), rand.randint(0,10), rand.randint(0,10)) for i in range(100)]

predictions = saved_model.predict(new_data)
for i in range(len(new_data)):
    print(f"month: {new_data[i][0]} temp: {new_data[i][1]} RH: {new_data[i][2]} wind: {new_data[i][3]} rain: {new_data[i][4]}\nchance of forest fire: {1 - predictions[i][0]:3f}\n")


4/4 [==============================] - 1s 9ms/step
month: 9 temp: 9 RH: 90 wind: 2 rain: 10
chance of forest fire: 0.011196

month: 1 temp: 25 RH: 79 wind: 1 rain: 3
chance of forest fire: 0.253325

month: 4 temp: 14 RH: 73 wind: 8 rain: 2
chance of forest fire: 0.261255

month: 3 temp: 35 RH: 45 wind: 2 rain: 6
chance of forest fire: 0.016395

month: 2 temp: 20 RH: 29 wind: 6 rain: 10
chance of forest fire: 0.018154

month: 5 temp: 24 RH: 23 wind: 9 rain: 2
chance of forest fire: 0.410449

month: 7 temp: 17 RH: 8 wind: 1 rain: 3
chance of forest fire: 0.245191

month: 4 temp: 26 RH: 52 wind: 2 rain: 4
chance of forest fire: 0.093226

month: 3 temp: 10 RH: 79 wind: 0 rain: 10
chance of forest fire: 0.075327

month: 7 temp: 2 RH: 96 wind: 10 rain: 10
chance of forest fire: 0.000118

month: 7 temp: 19 RH: 67 wind: 0 rain: 9
chance of forest fire: 0.034166

month: 7 temp: 24 RH: 88 wind: 10 rain: 7
chance of forest fire: 0.086990

month: 8 temp: 28 RH: 84 wind: 9 rain: 10
chance of forest

In [40]:
newdf = df[['month', 'temp', 'RH', 'wind', 'rain', 'area']]

In [42]:
newdf.corr()

,month,temp,RH,wind,rain,area
month,1.000000,0.368842,-0.095280,-0.086368,0.013438,0.056496
temp,0.368842,1.000000,-0.527390,-0.227116,0.069491,0.097844
RH,-0.095280,-0.527390,1.000000,0.069410,0.099751,-0.075519
wind,-0.086368,-0.227116,0.069410,1.000000,0.061119,0.012317
rain,0.013438,0.069491,0.099751,0.061119,1.000000,-0.007366
area,0.056496,0.097844,-0.075519,0.012317,-0.007366,1.000000


In [44]:
newdf.describe()

,month,temp,RH,wind,rain,area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,7.475822,18.889168,44.288201,4.017602,0.021663,12.847292
std,2.275990,5.806625,16.317469,1.791653,0.295959,63.655818
min,1.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,7.000000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,8.000000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,9.000000,22.800000,53.000000,4.900000,0.000000,6.570000
max,12.000000,33.300000,100.000000,9.400000,6.400000,1090.840000
